# Setup

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from generative_social_choice.utils.helper_functions import get_base_dir_path

In [16]:
from generative_social_choice.slates.voting_algorithms import (
    SequentialPhragmenMinimax,
    GreedyTotalUtilityMaximization,
    ExactTotalUtilityMaximization,
    LPTotalUtilityMaximization,
    VotingAlgorithm,
    ExactTotalUtilityMaximizationBruteSearch,
    GeometricTransformation,
)
from generative_social_choice.slates.voting_algorithm_axioms import (
    IndividualParetoAxiom,
    HappiestParetoAxiom,
    CoverageAxiom,
    MinimumAndTotalUtilityParetoAxiom,
    VotingAlgorithmAxiom,
    NonRadicalMinUtilityAxiom,
    NonRadicalTotalUtilityAxiom,
)
from generative_social_choice.test.utilities_for_testing import rated_vote_cases

# Load Data

In [17]:
voting_algorithm_evals_dir = get_base_dir_path() / "data" / "voting_algorithm_evals"
latest = True
if latest:
    file = sorted(voting_algorithm_evals_dir.glob("*.csv"))[-1]
else:
    file = voting_algorithm_evals_dir / "2025-11-07-234841.csv"
file

df = pd.read_csv(file, index_col=0, header=[0, 1])

In [18]:
file

WindowsPath('G:/Other computers/My Computer/NTFS/dev/generative_social_choice/generative_social_choice/data/voting_algorithm_evals/2025-12-17-223257.csv')

In [19]:
# Select subset of rows
# df = df.iloc[-8:]
df

vote                                                       Simple 1  \
subtest                                            Maximum Coverage   
ExactTotalUtilityMaximization(utility_transform...              1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...              1.0   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...                                    1.0   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...                          1.0   

vote                                                                                                    \
subtest                                            Minimum Utility and Total Utility Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                                                1.0    
ExactTotalUtilityMaximizationBruteSearch(utilit...                                                1.0    
ExactTotalUtilityMaximization(utility_transform...                                                1.0    
ExactTotalUtilityMaximizationBruteSearch(utilit...                                                1.0    

vote                                                                                            \
subtest                                            Non-radical Total Utility Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                                         1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...                                         1.0   
ExactTotalUtilityMaximization(utility_transform...                                         1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...                                         1.0   

vote                                                       Simple 2  \
subtest                                            Maximum Coverage   
ExactTotalUtilityMaximization(utility_transform...              1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...              1.0   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...                                    1.0   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
ExactTotalUtilityMaximizationBruteSearch(utilit...    

## Overall Performance

In [20]:
df.sum(axis=1)

ExactTotalUtilityMaximization(utility_transform=None)                                         142.208333
ExactTotalUtilityMaximizationBruteSearch(utility_transform=None)                              141.791667
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))               143.958333
ExactTotalUtilityMaximizationBruteSearch(utility_transform=GeometricTransformation(p=1.5))    143.916667
dtype: float64

In [21]:
pd.DataFrame(np.trunc(df.values + 0/24+1*1e-6), index=df.index, columns=df.columns).sum(axis=1)

ExactTotalUtilityMaximization(utility_transform=None)                                         139.0
ExactTotalUtilityMaximizationBruteSearch(utility_transform=None)                              138.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))               143.0
ExactTotalUtilityMaximizationBruteSearch(utility_transform=GeometricTransformation(p=1.5))    143.0
dtype: float64

## Differential Test Performance

In [28]:
# algs_for_non_uniform_rows = df
algs_for_non_uniform_rows = df.loc[[ExactTotalUtilityMaximization().name, ExactTotalUtilityMaximizationBruteSearch().name], :]
# algs_for_non_uniform_rows = df.loc[[ExactTotalUtilityMaximization(GeometricTransformation(p=1.5)).name, ExactTotalUtilityMaximizationBruteSearch(GeometricTransformation(p=1.5)).name], :]

non_uniform_columns = algs_for_non_uniform_rows.loc[:, algs_for_non_uniform_rows.nunique() > 1]
non_uniform_columns


vote,Ex 1.1 modified,Ex 4.1,Ex 4.3,Ex Alg2.1
subtest,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency
ExactTotalUtilityMaximization(utility_transform=None),0.625000,0.625000,1.000000,0.708333
ExactTotalUtilityMaximizationBruteSearch(utility_transform=None),0.291667,0.708333,0.958333,0.583333


In [29]:
non_uniform_columns.sum(axis=1)

ExactTotalUtilityMaximization(utility_transform=None)               2.958333
ExactTotalUtilityMaximizationBruteSearch(utility_transform=None)    2.541667
dtype: float64

These counts show which test cases and which axioms most frequently show unique behavior across the `VotingAlgorithm`s

In [30]:
# Count the number of columns with each name in both levels of the MultiIndex
level_0_counts = non_uniform_columns.columns.get_level_values(0).value_counts()
level_1_counts = non_uniform_columns.columns.get_level_values(1).value_counts()

print("Counts for level 0:")
print(level_0_counts)
print("\nCounts for level 1:")
print(level_1_counts)



Counts for level 0:
vote
Ex 1.1 modified    1
Ex 4.1             1
Ex 4.3             1
Ex Alg2.1          1
Name: count, dtype: int64

Counts for level 1:
subtest
Non-radical Total Utility Pareto Efficiency    4
Name: count, dtype: int64


## Axiom Pass Rate

In [31]:
df.groupby(df.columns.get_level_values(1), axis=1).mean()

C:\Users\aaron\AppData\Local\Temp\ipykernel_17456\1810388850.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df.groupby(df.columns.get_level_values(1), axis=1).mean()


subtest,Individual Pareto Efficiency,Maximum Coverage,Minimum Utility and Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,m-th Happiest Person Pareto Efficiency
ExactTotalUtilityMaximization(utility_transform=None),1.0,1.0,1.000000,0.903736,1.0
ExactTotalUtilityMaximizationBruteSearch(utility_transform=None),1.0,1.0,1.000000,0.889368,1.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.0,1.0,0.998563,0.965517,1.0
ExactTotalUtilityMaximizationBruteSearch(utility_transform=GeometricTransformation(p=1.5)),1.0,1.0,0.997126,0.965517,1.0


# Debuggign

In [50]:
# alg = GreedyTotalUtilityMaximization()
# alg = LPTotalUtilityMaximization()
# alg = SequentialPhragmenMinimax()
# alg = ExactTotalUtilityMaximization()
alg = ExactTotalUtilityMaximizationBruteSearch()
# alg = ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = SequentialPhragmenMinimax(load_magnitude_method="marginal_slate", clear_reassigned_loads=False, redistribute_defected_candidate_loads=False)

In [51]:
df1 = df.loc[alg.name,:].unstack()
df1

subtest,Individual Pareto Efficiency,Maximum Coverage,Minimum Utility and Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,m-th Happiest Person Pareto Efficiency
vote,,,,,
Ex 1.1,1.0,1.0,1.0,1.000000,1.0
Ex 1.1 modified,1.0,1.0,1.0,0.291667,1.0
Ex 1.2,1.0,1.0,1.0,1.000000,1.0
Ex 1.3,1.0,1.0,1.0,1.000000,1.0
Ex 2.1,1.0,1.0,1.0,1.000000,1.0
Ex 2.2,1.0,1.0,1.0,1.000000,1.0
Ex 3.1,1.0,1.0,1.0,1.000000,1.0
Ex 4.1,1.0,1.0,1.0,0.708333,1.0
Ex 4.2,1.0,1.0,1.0,1.000000,1.0


In [52]:
# case = rated_vote_cases["Ex 1.1 modified"]
from generative_social_choice.slates.voting_algorithms import RatedVoteCase
from generative_social_choice.slates.voting_utils import (
    CellWiseAugmentation, CandidateWiseAugmentation, VoterAndCellWiseAugmentation)


# case = rated_vote_cases["Ex A.1"]
# case = rated_vote_cases["Ex NRU1"]
case = rated_vote_cases["Ex 4.3"]

# case = RatedVoteCase(
#     name="Ex BIG",
#     rated_votes=case.rated_votes,
#     slate_size=case.slate_size,
#     noise_augmentation_methods = {
#                 CellWiseAugmentation(): 40, 
#                 CandidateWiseAugmentation(): 100, 
#                 VoterAndCellWiseAugmentation(): 100
#             }
# )
# case = rated_vote_cases["Ex 4.3"]
# case = rated_vote_cases["Simple 3"]
# axiom = CoverageAxiom()
# axiom = HappiestParetoAxiom()
# axiom = IndividualParetoAxiom()
# axiom = MinimumAndTotalUtilityParetoAxiom()
axiom = NonRadicalTotalUtilityAxiom()
# axiom = NonRadicalMinUtilityAxiom()

case.rated_votes.tail()

,s1,s2,s3,s4,s5
1,5,5,0,0,1
2,0,5,0,0,1
3,0,0,5,0,1
4,0,0,5,5,1
5,0,0,0,5,1


In [53]:
alg.name

'ExactTotalUtilityMaximizationBruteSearch(utility_transform=None)'

In [54]:
# df.loc[:,df.columns.get_level_values(1) == axiom.name].iloc[:,10:-10]
df.loc[alg.name,df.columns.get_level_values(1) == axiom.name]
# df.columns.get_level_values(1) == type(axiom).__name__
# df.columns.get_level_values(1), axiom.name


vote             subtest                                    
Simple 1         Non-radical Total Utility Pareto Efficiency    1.000000
Simple 2         Non-radical Total Utility Pareto Efficiency    1.000000
Simple 3         Non-radical Total Utility Pareto Efficiency    1.000000
Ex 1.1           Non-radical Total Utility Pareto Efficiency    1.000000
Ex 1.1 modified  Non-radical Total Utility Pareto Efficiency    0.291667
Ex 1.2           Non-radical Total Utility Pareto Efficiency    1.000000
Ex A.1           Non-radical Total Utility Pareto Efficiency    1.000000
Ex 1.3           Non-radical Total Utility Pareto Efficiency    1.000000
Ex 2.1           Non-radical Total Utility Pareto Efficiency    1.000000
Ex 2.2           Non-radical Total Utility Pareto Efficiency    1.000000
Ex 3.1           Non-radical Total Utility Pareto Efficiency    1.000000
Ex 4.1           Non-radical Total Utility Pareto Efficiency    0.708333
Ex 4.2           Non-radical Total Utility Pareto Efficiency   

In [55]:
alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)[0]

['s1', 's2', 's3']

In [56]:
# axiom.satisfactory_slates(rated_votes=case.rated_votes, slate_size=case.slate_size)
# case.augmented_cases[0]

In [57]:
num_aug_cases = len(case.augmented_cases)
aug_case_votes = pd.DataFrame(index = range(num_aug_cases), columns=["rated_votes", "assignments", "axiom_slates", "alg_slate", "satisfied"])
for i, aug_case in enumerate(case.augmented_cases):
    # print(aug_case)
    aug_case_votes.at[i, "rated_votes"] = aug_case
    aug_case_votes.at[i, "axiom_slates"] = axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "alg_slate"], aug_case_votes.at[i, "assignments"] = alg.vote(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "satisfied"] = axiom.evaluate_assignment(
        rated_votes=aug_case,
        assignments=aug_case_votes.at[i, "assignments"],
        slate_size=case.slate_size,
    )
    # print(axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size))
# aug_case_votes.iloc[:,2:]

In [58]:
failures = aug_case_votes[aug_case_votes["satisfied"] == False]
failures.iloc[:,2:]


,axiom_slates,alg_slate,satisfied
22,"{(s2, s1, s4), (s3, s5, s1), (s5, s1, s4)}","[s1, s2, s3]",False


In [59]:
f0 = failures.iloc[0]
f0.rated_votes
# (f0.rated_votes - case.rated_votes)*1e6


,s1,s2,s3,s4,s5
0,5.087325,0.071045,0.097178,0.076314,1.084317
1,5.130771,5.134130,0.111777,0.109448,1.098032
2,0.106053,5.136369,0.134490,0.134295,1.137147
3,0.118023,0.141150,5.104682,0.122996,1.107438
4,0.066768,0.091885,5.092163,5.068749,1.105078
5,0.100904,0.091361,0.102997,5.088866,1.094050


In [60]:
((f0.rated_votes - case.rated_votes)*1e6)

,s1,s2,s3,s4,s5
0,87324.657925,71045.341107,97178.232929,76313.526070,84317.273218
1,130770.919931,134130.107248,111777.328861,109447.978055,98032.235613
2,106052.897548,136368.514622,134489.670827,134294.905590,137146.755532
3,118023.030539,141150.461541,104682.110006,122995.759095,107437.741771
4,66767.809137,91885.450631,92163.372837,68748.552282,105077.607144
5,100903.965680,91361.155269,102996.814775,88866.377114,94049.841024


In [61]:
f0.assignments

,candidate_id,utility
0,s1,5.087325
1,s2,5.134130
2,s2,5.136369
3,s3,5.104682
4,s3,5.092163
5,s3,0.102997


In [62]:
alg.vote(rated_votes=f0.rated_votes, slate_size=case.slate_size)

(['s1', 's2', 's3'],
   candidate_id   utility
 0           s1  5.087325
 1           s2  5.134130
 2           s2  5.136369
 3           s3  5.104682
 4           s3  5.092163
 5           s3  0.102997)

In [65]:
axiom.evaluate_assignment(
    rated_votes=f0.rated_votes,
    assignments=f0.assignments,
    slate_size=case.slate_size,
)

False

In [64]:
from kiwiutils.kiwilib import leafClasses
[cls.__name__ for cls in leafClasses(VotingAlgorithmAxiom)]

['CoverageAxiom',
 'IndividualParetoAxiom',
 'NonRadicalMinUtilityAxiom',
 'NonRadicalTotalUtilityAxiom',
 'MinimumAndTotalUtilityParetoAxiom',
 'HappiestParetoAxiom']

# Scratch

In [48]:
alg1 = ExactTotalUtilityMaximization()
_, asg = alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)
asg

,candidate_id
0,s2
1,s2


In [49]:
axiom.evaluate_assignment(
    rated_votes=case.rated_votes,
    assignments=asg,
    slate_size=case.slate_size,
)

False